In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
df.columns

Index(['id', 'year', 'month', 'day', 'obs_date', 'season_type', 'avg_temp',
       'min_temp', 'max_temp', 'humidity', 'precipitation', 'wind_speed',
       'is_flood_risk', 'created_at', 'actual_flood', 'is_peak_rainy',
       'is_typhoon_season', 'is_early_rainy', 'precip_risk_level',
       'precip_ma3', 'humidity_ma3', 'precip_ma7', 'humidity_ma7',
       'rain_days_cumsum', 'data_quality_score', 'warning_risk_factor',
       'active_warnings', 'pressure', 'api_sources'],
      dtype='object')

In [4]:
df = pd.read_csv('C:/Project1/CREW_SOOM/data/raw/ML_COMPLETE_DATASET.csv')

#ML_COMPLETE_DATASET데이터셋의 컬럼들 중 필요한 12개 컬럼만 선택
feature_columns = [
    'precipitation', 'humidity', 'avg_temp', 'wind_speed',
            'month', 'is_peak_rainy', 'is_typhoon_season', 
            'precip_ma3', 'precip_ma7', 'rain_days_cumsum',
            'precip_risk_level', 'actual_flood'
]

#feature_columns 중 데이터에 존재하는 컬럼들만 선택
available_features = [col for col in feature_columns if col in df.columns]

#precipitation(강수량)은 홍수 예측에 필수
if 'precipitation' not in df.columns:
    print("❌ 기본 특성이 없습니다. 원본 데이터를 확인하세요.")

In [19]:
if 'is_flood_risk' in df.columns: #df에 is_flood_risk컬럼이 있을 경우 타겟으로 지정
    target_col = 'is_flood_risk'
else:
    #없을 경우 precipitation(강수량)이 50이상일 때를 타겟으로 지정하고 타입을 int로 변경
    df['is_flood_risk'] = (df['precipitation'] >= 50).astype(int) 
    target_col = 'is_flood_risk'

In [20]:
if 'month' not in df.columns and 'obs_date' in df.columns: #df에 obs_data컬럼은 있는데 month컬럼은 없을 경우
    df['month'] = pd.to_datetime(df['obs_date']).dt.month #obs_date를 datetime타입으로 변환 후 월만 추출
    
if 'is_peak_rainy' not in df.columns and 'month' in df.columns: #df에 month는 있는데 is_peak_rainy컬럼은 없을 경우
    df['is_peak_rainy'] = (df['month'].isin([6, 7])).astype(int) #df 'month'컬럼에 6, 7월이 있는지 확인하고 확인 여부를 int로 형변환
    

is_flood_risk


In [22]:
(df['month'].isin([6, 7])).astype(int)

0      0
1      0
2      0
3      0
4      0
      ..
928    1
929    1
930    1
931    1
932    1
Name: month, Length: 933, dtype: int32

In [25]:
df['season_type'].value_counts

<bound method IndexOpsMixin.value_counts of 0        dry
1        dry
2        dry
3        dry
4        dry
       ...  
928    rainy
929    rainy
930    rainy
931    rainy
932    rainy
Name: season_type, Length: 933, dtype: object>

In [37]:
#최종 컬럼 선택
final_features = [col for col in available_features if col!='actual_flood']
#available_features중 'actual_flood'컬럼 제외 모두 final_features에 넣기
final_features = [col for col in final_features if col in df.columns]
#final_features컬럼 중 df컬럼에 있는 컬럼들로 업데이트

X = df[final_features] #X는 final_features를 컬럼으로 하는 데이터프레임
y = df[target_col] #y는 target_col을 컬럼으로 하는 데이터프레임
print(X, y)
type(X)

     precipitation  humidity  avg_temp  wind_speed  month  is_peak_rainy  \
0              0.0      46.3      -4.3         1.5      1              0   
1              0.3      57.9      -1.3         2.3      1              0   
2              0.0      58.3      -1.9         1.8      1              0   
3              0.0      51.8      -2.5         2.4      1              0   
4              0.0      47.6      -2.8         1.7      1              0   
..             ...       ...       ...         ...    ...            ...   
928            8.0      88.9      20.9         2.5      6              1   
929            1.1      85.3      22.2         1.7      6              1   
930            0.0      78.1      25.0         1.4      6              1   
931            0.6      85.0      26.2         1.4      6              1   
932            0.0      77.1      27.1         2.3      6              1   

     is_typhoon_season  precip_ma3  precip_ma7  rain_days_cumsum  \
0                  

pandas.core.frame.DataFrame

In [40]:
X.info() #wind_speed에 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   precipitation      933 non-null    float64
 1   humidity           933 non-null    float64
 2   avg_temp           933 non-null    float64
 3   wind_speed         931 non-null    float64
 4   month              933 non-null    int64  
 5   is_peak_rainy      933 non-null    int64  
 6   is_typhoon_season  933 non-null    int64  
 7   precip_ma3         933 non-null    float64
 8   precip_ma7         933 non-null    float64
 9   rain_days_cumsum   933 non-null    int64  
 10  precip_risk_level  933 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 80.3 KB


In [41]:
X = X.fillna(X.median()) #중앙값으로 결측치 처리

In [43]:
print(f"✅ 특성 준비 완료:")
print(f"   - 특성 수: {len(final_features)}")
print(f"   - 샘플 수: {len(X)}")
print(f"   - 양성 비율: {y.mean()*100:.1f}%")

✅ 특성 준비 완료:
   - 특성 수: 11
   - 샘플 수: 933
   - 양성 비율: 2.5%


# 데이터 전처리 클래스

In [ ]:
class DataPreprocessor:
    def __init__(self): #생성자함수(클래스 호출 시 자동실행)
        self.scaler = StandardScaler() #평균0, 표준편차1로 정규화
        
    def prepare_features(self, df):
        """특성 준비 (기존 ML_COMPLETE_DATASET 활용)"""
        #필요한 특성만 선택
        feature_columns = [
            'precipitation', 'humidity', 'avg_temp', 'wind_speed',
            'month', 'is_peak_rainy', 'is_typhoon_season', 
            'precip_ma3', 'precip_ma7', 'rain_days_cumsum',
            'precip_risk_level', 'actual_flood'
        ]
        
        #이중 df에 실제로 존재하는 컬럼만 가져오기
        available_features = [col for col in feature_columns if col in df.columns]
        
        #침수 위험 예측에 필수적인 강수량(precipitation)이 없으면 경고
        if 'precipitation' is not in df.columns:
            print("❌ 기본 특성이 없습니다. 원본 데이터를 확인하세요.")
            return None, None, None #X, y, final_features 모두 반환X
        
        if 'is_flood_risk' in df.columns: # 침수 위험 컬럼이 있으면 타겟 컬럼으로 지정
            target_col = 'is_flood_risk'
        else: #없으면 50mm이상을 침수 위험이라 지정하고 정수로 형변환 후 타겟 컬럼으로 지정
            df['is_flood_risk'] = (df['precipitation']>=50).astype(int) 
        
        #필수 특성(월 컬럼)이 없는 경우 생성
        if 'month' not in df.columns:
            df['month'] = pd.to_datetime(df['obs_date']).dt.month #obs_data를 datetime타입으로 바꿔서 월만 뽑은 후 month컬럼에 넣기
            
        if 'is_peak_rainy' is not in df.columns and 'month' in df.columns: #month가 있고, 비가 많이 오는날 컬럼이 없을 때
            df['is_peak_rainy'] = (df['month'].isin([6,7])).astype(int) #6, 7월을 peak_rainy로 지정하고 int로 형변환
        
        #최종 특성 선택
        final_features = [col for col in available_features if col !='actual_flood'] #컬럼이 실제 홍수 발생 여부가 아니면 최종 특성에 추가
        final_features = [col for col in final_features if col in df.columns]
        
        X = df[final_features] #X=df의 최종 특성
        y = df[target_col] #y=df의 타겟 컬럼 데이터(is_flood_risk)
        
        #결측치를 중앙값으로 처리
        X = X.fillna(X.median)
        #y는 결측치 없음
        
        print(f"✅ 특성 준비 완료:")
        print(f"   - 특성 수: {len(final_features)}")
        print(f"   - 샘플 수: {len(X)}")
        print(f"   - 양성 비율: {y.mean()*100:.1f}%")
        
        return X, y, final_features